## 1. Preparación de los Datos

Consulta Ajustada

Dado que deseas capturar la estacionalidad, es necesario incluir datos del mismo mes del año anterior. Asegúrate de incluir el rango temporal necesario en la consulta:

### FECHA_CORTE: Debe ser 30 días a la fecha actual --> La vamos a usar como límite del entrenamiento del modelo
### FECHA_FUTURA: Vamos a usar los 30 días reales para comparar contra el real

In [1]:
FECHA_CORTE = '20241130'
FECHA_FUTURA = '20241230'

#### a) Para Funcionas asegurarse de conectarse a la VPN

Pass: Z33treex.2024	

DIARCOEST -->  SQLSERVER:192.168.0.250   eettlin/connexa.2024

In [2]:
# OBTENER PARÁMETROS de la CONEXIÓN

import pyodbc
from dotenv import dotenv_values

secrets = dotenv_values(".env")
local_secrets = dotenv_values(".env.dev")

DRIVER = secrets["DRIVER"]
SERVIDOR = secrets["SERVIDOR"]
PUERTO = secrets["PUERTO"]
BASE = secrets["BASE"]
USUARIO = secrets["USUARIO"]
CONTRASENA = secrets["CONTRASENA"]


constr = f'DRIVER={DRIVER};SERVER ={SERVIDOR};BASE={BASE};USER={USUARIO};PWD={CONTRASENA}'

print (constr)


DRIVER=ODBC Driver 17 for SQL Server;SERVER =192.168.0.250;BASE=DiarcoEst;USER=eettlin;PWD=connexa.2024


In [4]:
import pandas as pd
import pyodbc

# Configuración de conexión
constr = f'DRIVER={DRIVER};SERVER ={SERVIDOR};PORT={PUERTO};BASE={BASE};USER={USUARIO};PWD={CONTRASENA}'

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=192.168.0.250;"
     "PORT=1433;"
    "DATABASE=DiarcoEst;"
    "UID=eettlin;"
    "PWD=connexa.2024;"
)
conn = pyodbc.connect(conn_str)

query = """
-- Consulta con datos históricos y actuales
SELECT V.[F_VENTA],
       V.[C_ARTICULO],
       V.[C_SUCU_EMPR],
       V.[Q_UNIDADES_VENDIDAS],
       V.[I_PRECIO_VENTA],
       V.[I_PRECIO_COSTO],
       V.[I_VENDIDO],
       V.[I_PRECIO_COSTO_PP],
       A.[C_FAMILIA],
       A.[C_RUBRO],
       A.[C_SUBRUBRO_1],
       A.[C_SUBRUBRO_2],
       A.[N_ARTICULO]
FROM [DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO] V
LEFT JOIN [DiarcoEst].[dbo].[T050_ARTICULOS] A 
    ON V.C_ARTICULO = A.C_ARTICULO
WHERE V.C_ARTICULO BETWEEN  50 AND 200  AND V.F_VENTA >= DATEADD(YEAR, -1, '20241130')
"""
data = pd.read_sql(query, conn)
conn.close()


C:\Users\eduar\AppData\Local\Temp\ipykernel_34384\1206587002.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, conn)


### 2. Ingeniería de Características
Una vez cargados los datos, realiza los siguientes pasos para crear características relevantes:

In [5]:
#### 2.1 Variables Temporales
# Extrae información adicional de la columna F_VENTA:

data['F_VENTA'] = pd.to_datetime(data['F_VENTA'])
data['mes'] = data['F_VENTA'].dt.month
data['dia_semana'] = data['F_VENTA'].dt.dayofweek
data['es_fin_semana'] = data['dia_semana'].isin([5, 6]).astype(int)
data['año'] = data['F_VENTA'].dt.year


# 2.2 Precio Relativo
# Calcula el precio relativo respecto al promedio histórico del artículo:

data = data[data['I_PRECIO_COSTO_PP'] != 0]    # Filtrar Datos sin Costo para que no de Overflow
data['precio_relativo'] = data['I_PRECIO_VENTA'] / data['I_PRECIO_COSTO_PP']

# 2.3 Estacionalidad
# Agrupa las ventas por mes para detectar tendencias estacionales:

estacionalidad = data.groupby(['mes', 'C_ARTICULO'])['Q_UNIDADES_VENDIDAS'].mean().reset_index()
data = pd.merge(data, estacionalidad, on=['mes', 'C_ARTICULO'], suffixes=('', '_estacional'))

#2.4 Variables Categóricas
#Convierte las categorías en variables dummy:

data = pd.get_dummies(data, columns=['C_FAMILIA', 'C_RUBRO', 'C_SUBRUBRO_1', 'C_SUBRUBRO_2'], drop_first=True)


## 3. Modelado
Probaremos varios algoritmos para estimar la demanda.

### 3.1 Dividir Datos

Define las variables independientes ( 𝑋 ) y dependientes (𝑦):

In [6]:
X = data[['precio_relativo', 'es_fin_semana', 'dia_semana', 'Q_UNIDADES_VENDIDAS_estacional']]
y = data['Q_UNIDADES_VENDIDAS']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### REVISAR CALIDAD de los DATOS

In [7]:
import numpy as np

# Verificar NaN
print("Valores NaN en X_train:", np.any(np.isnan(X_train)))

# Verificar Inf
print("Valores Inf en X_train:", np.any(np.isinf(X_train)))

# Verificar valores extremos
print("Valores máximos en X_train:", np.max(X_train, axis=0))
print("Valores mínimos en X_train:", np.min(X_train, axis=0))

Valores NaN en X_train: False
Valores Inf en X_train: False
Valores máximos en X_train: precio_relativo                   7380.299000
es_fin_semana                        1.000000
dia_semana                           6.000000
Q_UNIDADES_VENDIDAS_estacional     213.384615
dtype: float64
Valores mínimos en X_train: precio_relativo                   0.245946
es_fin_semana                     0.000000
dia_semana                        0.000000
Q_UNIDADES_VENDIDAS_estacional    0.000000
dtype: float64


### Ajustar los Datos si hay Errores

In [8]:
from sklearn.preprocessing import StandardScaler

# Revisar y manejar valores problemáticos
X_train = np.nan_to_num(X_train, nan=np.nanmean(X_train, axis=0))  # Rellenar NaN
X_train = np.where(np.isinf(X_train), np.nanmax(X_train), X_train)  # Rellenar Inf

# Normalizar valores grandes
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

### Ultimo Control de Todo Bien 

In [9]:
# Validar que no haya valores NaN ni Inf en X_train
assert not np.any(np.isnan(X_train)), "Aún hay valores NaN en X_train"
assert not np.any(np.isinf(X_train)), "Aún hay valores Inf en X_train"


### 3.2 Modelos a Probar

#### Regresión Lineal


In [10]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

e:\PY\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


#### Random Forest

In [11]:
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

e:\PY\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


### XGBoost

In [12]:
from xgboost import XGBRegressor
model_xgb = XGBRegressor(random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)


### 3.3 Evaluación de los Modelos
Compara los resultados usando métricas como RMSE o 𝑅 2R 2
 :

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def evaluar_modelo(y_test, y_pred, nombre):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{nombre} - RMSE: {rmse:.2f}, R^2: {r2:.2f}")

evaluar_modelo(y_test, y_pred_lr, "Regresión Lineal")
evaluar_modelo(y_test, y_pred_rf, "Random Forest")
evaluar_modelo(y_test, y_pred_xgb, "XGBoost")


Regresión Lineal - RMSE: 1650.50, R^2: -117.72
Random Forest - RMSE: 152.29, R^2: -0.01
XGBoost - RMSE: 542.02, R^2: -11.80


## 4. Predicción de Demanda Futura

Una vez que el mejor modelo esté identificado, puedes usarlo para predecir la demanda futura generando un conjunto de datos con las fechas próximas y las características calculadas de manera similar:

In [21]:
# Crear un DataFrame de fechas futuras
future_data = pd.DataFrame({
    'F_VENTA': pd.date_range(start='2024-12-01', periods=30, freq='D'),
    'mes': pd.date_range(start='2024-12-01', periods=30, freq='D').month,
    'dia_semana': pd.date_range(start='2024-12-01', periods=30, freq='D').dayofweek,
})
future_data['es_fin_semana'] = future_data['dia_semana'].isin([5, 6]).astype(int)

# Asociar datos futuros con los artículos (puedes duplicar filas para cada artículo)
future_data = future_data.assign(C_ARTICULO=data['C_ARTICULO'].unique()[0])  # Cambiar para manejar múltiples artículos

# Calcular precio relativo (basado en datos históricos)
future_data = future_data.merge(
    data.groupby('C_ARTICULO')['I_PRECIO_COSTO_PP'].mean().reset_index().rename(columns={'I_PRECIO_COSTO_PP': 'precio_regular'}),
    on='C_ARTICULO',
    how='left'
)
future_data['I_PRECIO_VENTA'] = data.groupby('C_ARTICULO')['I_PRECIO_VENTA'].transform('mean')
future_data['precio_relativo'] = future_data['I_PRECIO_VENTA'] / future_data['precio_regular']

# Asociar estacionalidad
estacionalidad = data.groupby(['mes', 'C_ARTICULO'])['Q_UNIDADES_VENDIDAS'].mean().reset_index()
estacionalidad.rename(columns={'Q_UNIDADES_VENDIDAS': 'Q_UNIDADES_VENDIDAS_estacional'}, inplace=True)
future_data = pd.merge(future_data, estacionalidad, on=['mes', 'C_ARTICULO'], how='left')

# Realizar predicciones
future_data['pred_demanda'] = model_rf.predict(future_data[['precio_relativo', 'es_fin_semana', 'dia_semana', 'Q_UNIDADES_VENDIDAS_estacional']])

# Mostrar resultados
print(future_data[['F_VENTA', 'C_ARTICULO', 'pred_demanda','I_PRECIO_VENTA']])


      F_VENTA  C_ARTICULO  pred_demanda  I_PRECIO_VENTA
0  2024-12-01        52.0     39.736833     1242.433052
1  2024-12-02        52.0     30.066667     1242.433052
2  2024-12-03        52.0     10.753667     1242.433052
3  2024-12-04        52.0     14.481667     1242.433052
4  2024-12-05        52.0     14.481667     1242.433052
5  2024-12-06        52.0     14.481667     1242.433052
6  2024-12-07        52.0     39.736833     1242.433052
7  2024-12-08        52.0     39.736833     1242.433052
8  2024-12-09        52.0     30.066667     1242.433052
9  2024-12-10        52.0     10.753667     1242.433052
10 2024-12-11        52.0     14.481667     1242.433052
11 2024-12-12        52.0     14.481667     1242.433052
12 2024-12-13        52.0     14.481667     1242.433052
13 2024-12-14        52.0     39.736833     1242.433052
14 2024-12-15        52.0     39.736833     1242.433052
15 2024-12-16        52.0     30.066667     1242.433052
16 2024-12-17        52.0     10.753667     1242

e:\PY\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


### SIMULARIÓN COMPLETA

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Calcular precio relativo promedio por artículo
precios = data.groupby('C_ARTICULO')[['I_PRECIO_COSTO_PP', 'I_PRECIO_VENTA']].mean().reset_index()
precios.rename(columns={'I_PRECIO_COSTO_PP': 'precio_regular', 'I_PRECIO_VENTA': 'precio_promedio'}, inplace=True)

# Asociar precios al conjunto futuro
future_data = pd.merge(future_data, precios, on='C_ARTICULO', how='left')
future_data['precio_relativo'] = future_data['precio_promedio'] / future_data['precio_regular']

# 5. Predicción de la demanda futura
X_future = future_data[['precio_relativo', 'es_fin_semana', 'dia_semana', 'mes']]
future_data['pred_demanda'] = model_rf.predict(X_future)

# 6. Comparar con datos reales
query_real = """
    SELECT F_VENTA, C_ARTICULO, Q_UNIDADES_VENDIDAS
    FROM [DiarcoEst].[dbo].[T702_EST_VTAS_POR_ARTICULO]
    WHERE C_ARTICULO BETWEEN  50 AND 100  AND 
        F_VENTA BETWEEN CONVERT(DATE, '20241130') AND DATEADD(DAY, 30, CONVERT(DATE, '20241130'))
"""
real_data = pd.read_sql(query_real, conn)
comparacion = pd.merge(future_data, real_data, on=['F_VENTA', 'C_ARTICULO'], how='left')
comparacion['error'] = comparacion['pred_demanda'] - comparacion['Q_UNIDADES_VENDIDAS']

# 7. Métricas de evaluación
mae = mean_absolute_error(comparacion['Q_UNIDADES_VENDIDAS'].dropna(), comparacion['pred_demanda'])
rmse = np.sqrt(mean_squared_error(comparacion['Q_UNIDADES_VENDIDAS'].dropna(), comparacion['pred_demanda']))
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# 8. Visualización de resultados
articulo_especifico = data['C_ARTICULO'].unique()[0]
datos_articulo = comparacion[comparacion['C_ARTICULO'] == articulo_especifico]

plt.figure(figsize=(10, 6))
plt.plot(datos_articulo['F_VENTA'], datos_articulo['pred_demanda'], label='Predicción', marker='o')
plt.plot(datos_articulo['F_VENTA'], datos_articulo['Q_UNIDADES_VENDIDAS'], label='Real', marker='x')
plt.title(f'Predicción vs Real para Artículo {articulo_especifico}')
plt.xlabel('Fecha')
plt.ylabel('Demanda')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.show()


MergeError: Passing 'suffixes' which cause duplicate columns {'precio_regular_x'} is not allowed.